In [65]:
import pinocchio
import numpy as np 
import time 
import pymanopt 
from pymanopt import manifolds, optimizers, tools, core
from pymanopt.core.problem  import Problem

from utils import *
import networkx as nx
from scipy import optimize
from scipy.optimize import Bounds

LOWER = "lower_limit"
UPPER = "upper_limit"
BOUNDED = "bounded"
BELOW = "below"
ABOVE = "above"
TYPE = "type"
OBSTACLE = "obstacle"
ROBOT = "robot"
END_EFFECTOR = "end_effector"
RADIUS = "radius"
DIST = "weight"
POS = "pos"
BASE = "base"
ANCHOR = "anchor"
UNDEFINED = None

deg2rad = np.pi/180.
rad2deg = 180./np.pi

scale = 1

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [243]:
def create_base_graph(model,data,axis_length):

    trans_z = pinocchio.SE3(np.eye(3),np.array([0,0,axis_length]))

    q_init = pinocchio.neutral(model)
    pinocchio.forwardKinematics(model,data,q_init)
    
    robot_name = model.names[1]
    ROOT = robot_name

    base = nx.empty_graph()

    for idx, (name, oMi) in enumerate(zip(model.names, data.oMi)):
        if idx > 1:
            cur, aux_cur = (name, name+'_tilde')
            cur_pos, aux_cur_pos = (
                oMi.translation*scale,
                pinocchio.SE3.act(pinocchio.SE3(oMi.rotation,oMi.translation*scale),trans_z).translation,
            )
            print(name,cur_pos,aux_cur_pos)
            dist = np.linalg.norm(cur_pos - aux_cur_pos)

            # Add nodes for joint and edge between them
            base.add_nodes_from(
                [(cur, {POS: cur_pos}), (aux_cur, {POS: aux_cur_pos})]
            )
            base.add_edge(
                cur, aux_cur, **{DIST: dist, LOWER: dist, UPPER: dist, BOUNDED: [], ANCHOR: False}
            )

            # If there exists a preceeding joint, connect it to new
            if idx != 0:
                pred, aux_pred = (model.names[idx-1], model.names[idx-1]+'_tilde')
                for u in [pred, aux_pred]:
                    for v in [cur, aux_cur]:
                        dist = np.linalg.norm(
                            base.nodes[u][POS] - base.nodes[v][POS]
                        )
                        base.add_edge(
                            u,
                            v,
                            **{DIST: dist, LOWER: dist, UPPER: dist, BOUNDED: [], ANCHOR: False},
                        )
        elif idx == 1:
            print(name,oMi.translation)
            base = nx.DiGraph(
                [
                    (robot_name, "x"),
                    (robot_name, "y"),
                    (robot_name, robot_name+'_tilde'),
                    ("x", "y"),
                    ("y", robot_name+'_tilde'),
                    (robot_name+'_tilde' ,"x"),
                ]
            )
            base.add_nodes_from(
                [
                    ("x", {POS: np.array([axis_length, 0, 0]) + oMi.translation*scale, TYPE: [BASE]}),
                    ("y", {POS: np.array([0, -axis_length, 0]) + oMi.translation*scale, TYPE: [BASE]}),
                    (robot_name, {POS: oMi.translation*scale, TYPE: [ROBOT, BASE]}),
                    (robot_name+'_tilde', {POS: np.array([0, 0, axis_length]) + oMi.translation*scale, TYPE: [ROBOT, BASE]}),
                ]
            )
            for u, v in base.edges():
                base[u][v][DIST] = np.linalg.norm(base.nodes[u][POS] - base.nodes[v][POS])
                base[u][v][LOWER] = base[u][v][DIST]
                base[u][v][UPPER] = base[u][v][DIST]
                base[u][v][ANCHOR] = True
                base[u][v][BOUNDED] = []

    # Set node type to robot
    nx.set_node_attributes(base, [ROBOT], TYPE)
    base.nodes[ROOT][TYPE] = [ROBOT, BASE]
    base.nodes[ROOT + '_tilde'][TYPE] = [ROBOT, BASE]

    return base

In [244]:
axis_length = 0.2*scale
urdf_filename="GraphIK/graphik/robots/urdfs/kuka_iiwr.urdf"

model    = pinocchio.buildModelFromUrdf(urdf_filename)
data     = model.createData()

G = create_base_graph(model,data,axis_length)


lbr_iiwa_joint_1 [0.     0.     0.1575]
lbr_iiwa_joint_2 [0.   0.   0.36] [-4.13646214e-14  2.00000000e-01  3.60000000e-01]
lbr_iiwa_joint_3 [ 2.07103964e-25 -1.00133990e-12  5.64500000e-01] [ 4.13646214e-14 -1.00131770e-12  7.64500000e-01]
lbr_iiwa_joint_4 [ 4.45703796e-14 -1.00131597e-12  7.80000000e-01] [ 8.5935001e-14 -2.0000000e-01  7.8000000e-01]
lbr_iiwa_joint_5 [ 8.27292428e-14 -9.78656600e-14  9.64500000e-01] [1.24093864e-13 1.86078980e-12 1.16450000e+00]
lbr_iiwa_joint_6 [1.27299622e-13 2.01258560e-12 1.18000000e+00] [4.45703796e-14 2.00000000e-01 1.18000000e+00]
lbr_iiwa_joint_7 [1.44052294e-13 2.40921333e-12 1.26100000e+00] [1.85416916e-13 2.40923553e-12 1.46100000e+00]


In [245]:
nx.get_node_attributes(G,POS)

{'lbr_iiwa_joint_1': array([0.    , 0.    , 0.1575]),
 'x': array([0.2   , 0.    , 0.1575]),
 'y': array([ 0.    , -0.2   ,  0.1575]),
 'lbr_iiwa_joint_1_tilde': array([0.    , 0.    , 0.3575]),
 'lbr_iiwa_joint_2': array([0.  , 0.  , 0.36]),
 'lbr_iiwa_joint_2_tilde': array([-4.13646214e-14,  2.00000000e-01,  3.60000000e-01]),
 'lbr_iiwa_joint_3': array([ 2.07103964e-25, -1.00133990e-12,  5.64500000e-01]),
 'lbr_iiwa_joint_3_tilde': array([ 4.13646214e-14, -1.00131770e-12,  7.64500000e-01]),
 'lbr_iiwa_joint_4': array([ 4.45703796e-14, -1.00131597e-12,  7.80000000e-01]),
 'lbr_iiwa_joint_4_tilde': array([ 8.5935001e-14, -2.0000000e-01,  7.8000000e-01]),
 'lbr_iiwa_joint_5': array([ 8.27292428e-14, -9.78656600e-14,  9.64500000e-01]),
 'lbr_iiwa_joint_5_tilde': array([1.24093864e-13, 1.86078980e-12, 1.16450000e+00]),
 'lbr_iiwa_joint_6': array([1.27299622e-13, 2.01258560e-12, 1.18000000e+00]),
 'lbr_iiwa_joint_6_tilde': array([4.45703796e-14, 2.00000000e-01, 1.18000000e+00]),
 'lbr_iiwa

In [246]:

def norm(x):
    return x/max(np.linalg.norm(x),10**-9)

def goal_graph(model, data, G, axis_length, position, direction = None, anchor = True):
    
    end_effector = [model.names[-1]]
    if direction is not None:
        end_effector.append(model.names[-1]+'_tilde')

    list_nodes_base = ['x','y',model.names[1],model.names[1]+'_tilde']

    for i, cur_end_effector in enumerate(end_effector):
        pos_end = position

        if direction is not None and i == 1:
            pos_end = pos_end + direction*axis_length
        
        G.nodes[cur_end_effector][POS] = pos_end

        for cur_base in list_nodes_base:
            
            if not((i == 1) and (len(end_effector)==1)):
                dist = np.linalg.norm(pos_end-G.nodes[cur_base][POS])

                G.add_edge(
                            cur_base, cur_end_effector, **{DIST: dist, LOWER: dist, UPPER: dist, BOUNDED: [], ANCHOR: anchor}
                        )
    
    if anchor:

        list_edges_anchor = []
        list_other_edges = []
        new_edge_order  = []
        A_anchor = []

        for key ,anch_ in nx.get_edge_attributes(G,ANCHOR).items():

            if anch_:
                list_edges_anchor.append(key)
                A_anchor.append(norm(G.nodes[key[1]][POS]- G.nodes[key[0]][POS]))
                print(key,' : ',norm(G.nodes[key[1]][POS]- G.nodes[key[0]][POS]))
            else:
                list_other_edges.append(key)
        new_edge_order = list_edges_anchor.copy() #.extend(list_other_edges)

        for e in list_other_edges:
            new_edge_order.append(e)

        A_anchor = np.array(A_anchor).transpose()
        
    else:

        list_edges_anchor = []
        list_other_edges = [e for e in G.edges()]

        new_edge_order = list_other_edges
        A_anchor = []

    n = len(list_other_edges)
    n_anchor = len(list_edges_anchor)

    weight = nx.get_edge_attributes(G, DIST)
    D = np.diag([weight[i] for i in new_edge_order])

    C = incidence_matrix_(G,oriented=True, edgelist=new_edge_order).toarray()
    C = np.array(C)

    Y_init = np.zeros((3,n))

    for i, e in enumerate(list_other_edges):
        i = i 
        p1 = G.nodes[e[0]][POS]
        p2 = G.nodes[e[1]][POS]
        Y_init[:,i] = (p2-p1)/max(np.linalg.norm(p2-p1),10**-9)

    if anchor:
        Y_init = [A_anchor,Y_init]
    return G, list_edges_anchor, list_edges_anchor, new_edge_order, n,n_anchor, A_anchor, D,C, Y_init

In [247]:
q0 = np.array([0,0,0,0,0,0,0])*np.pi/180
pinocchio.framesForwardKinematics(model,data,q0)
position = data.oMi[-1].translation*scale #np.array([200,0,500])/1000
direction = data.oMi[-1].rotation@np.array([0,0,1])
print(direction)
data.oMi[7].rotation

[2.06823107e-13 1.11022303e-16 1.00000000e+00]


array([[ 1.00000000e+00, -6.20469321e-13,  2.06823107e-13],
       [ 6.20469321e-13,  1.00000000e+00,  1.11022303e-16],
       [-2.06823107e-13,  1.11022303e-16,  1.00000000e+00]])

In [248]:
q0 = np.array([30,75,10,-40,90,0,0])*np.pi/180
q0 = np.array([0,0,0,0,0,0,0])*np.pi/180
pinocchio.framesForwardKinematics(model,data,q0)
position = data.oMi[-1].translation*scale #np.array([200,0,500])/1000
direction = data.oMi[-1].rotation@np.array([0,0,1])

print(position, direction)

G1, list_edges_anchor, list_edges_anchor, new_edge_order, n,n_anchor, A_anchor, D,C, Y_init_ = goal_graph(model, data, G, axis_length, 
                                                                                                 position, direction = direction, anchor = True)




[1.44052294e-13 2.40921333e-12 1.26100000e+00] [2.06823107e-13 1.11022303e-16 1.00000000e+00]
('lbr_iiwa_joint_1', 'x')  :  [1. 0. 0.]
('lbr_iiwa_joint_1', 'y')  :  [ 0. -1.  0.]
('lbr_iiwa_joint_1', 'lbr_iiwa_joint_1_tilde')  :  [0. 0. 1.]
('lbr_iiwa_joint_1', 'lbr_iiwa_joint_7')  :  [1.30541272e-13 2.18324724e-12 1.00000000e+00]
('lbr_iiwa_joint_1', 'lbr_iiwa_joint_7_tilde')  :  [1.42245428e-13 1.84828196e-12 1.00000000e+00]
('x', 'y')  :  [-0.70710678 -0.70710678  0.        ]
('x', 'lbr_iiwa_joint_7')  :  [-1.78336135e-01  2.14824897e-12  9.83969625e-01]
('x', 'lbr_iiwa_joint_7_tilde')  :  [-1.51658302e-01  1.82690285e-12  9.88432982e-01]
('y', 'lbr_iiwa_joint_1_tilde')  :  [0.         0.70710678 0.70710678]
('y', 'lbr_iiwa_joint_7')  :  [1.28448647e-13 1.78336135e-01 9.83969625e-01]
('y', 'lbr_iiwa_joint_7_tilde')  :  [1.40600073e-13 1.51658302e-01 9.88432982e-01]
('lbr_iiwa_joint_1_tilde', 'x')  :  [ 0.70710678  0.         -0.70710678]
('lbr_iiwa_joint_1_tilde', 'lbr_iiwa_joint_7'

In [249]:
nx.get_node_attributes(G1,POS)

{'lbr_iiwa_joint_1': array([0.    , 0.    , 0.1575]),
 'x': array([0.2   , 0.    , 0.1575]),
 'y': array([ 0.    , -0.2   ,  0.1575]),
 'lbr_iiwa_joint_1_tilde': array([0.    , 0.    , 0.3575]),
 'lbr_iiwa_joint_2': array([0.  , 0.  , 0.36]),
 'lbr_iiwa_joint_2_tilde': array([-4.13646214e-14,  2.00000000e-01,  3.60000000e-01]),
 'lbr_iiwa_joint_3': array([ 2.07103964e-25, -1.00133990e-12,  5.64500000e-01]),
 'lbr_iiwa_joint_3_tilde': array([ 4.13646214e-14, -1.00131770e-12,  7.64500000e-01]),
 'lbr_iiwa_joint_4': array([ 4.45703796e-14, -1.00131597e-12,  7.80000000e-01]),
 'lbr_iiwa_joint_4_tilde': array([ 8.5935001e-14, -2.0000000e-01,  7.8000000e-01]),
 'lbr_iiwa_joint_5': array([ 8.27292428e-14, -9.78656600e-14,  9.64500000e-01]),
 'lbr_iiwa_joint_5_tilde': array([1.24093864e-13, 1.86078980e-12, 1.16450000e+00]),
 'lbr_iiwa_joint_6': array([1.27299622e-13, 2.01258560e-12, 1.18000000e+00]),
 'lbr_iiwa_joint_6_tilde': array([4.45703796e-14, 2.00000000e-01, 1.18000000e+00]),
 'lbr_iiwa

In [250]:
def simple_IK(d,C,D,A_,n, n_anchor,max_iter,ind=None,W=None,Y_init=None,use_rand=False):
    if  W is None:
        W = np.eye(D.shape[0])
    if n_anchor > 0:    
        if d > 3:
            A = np.zeros((d,n_anchor))
            A[:3,:] = A_[:3,:]
        else:
            if A_.shape[0] == d:
                A = A_
            elif A_.shape[0] > d:
                A = A_[:d,:]
            else:
                A = np.zeros((d,n_anchor))
                A[:3,:] = A_[:3,:]

        manifold_ = manifolds.Product([manifolds.ConstantFactory(A) , manifolds.Oblique(d,n)])
    else:
        manifold_ = manifolds.Oblique(d,n)

    Q2 = -D@(W@C.transpose()@np.linalg.pinv(C@W@C.transpose()))@C@W@D
    Q1 = D@W@D

    if n_anchor > 0:
        zero = np.zeros_like(A)
        @pymanopt.function.numpy(manifold_)
        def cost(A,Y_):
            print(A.shape,Y_.shape)
            Y = np.concatenate([A,Y_],axis=1)
            return np.trace(Q1 + Q2@Y.transpose()@Y) 

        @pymanopt.function.numpy(manifold_)
        def euclidean_gradient(A,Y_):
            Y = np.concatenate([A,Y_],axis=1)
            grad = 2*Q2[n_anchor:,:]@Y.transpose()
            return [zero,grad.transpose() ]
                                
        @pymanopt.function.numpy(manifold_)
        def euclidean_hessian(A,Y_,U_A, U):
            U_ = np.concatenate([U_A,U],axis=1)
            hess = 2*Q2[n_anchor:,:]@U_.transpose()
            return [zero,hess.transpose()]
    else:
        @pymanopt.function.numpy(manifold_)
        def cost(Y):
            return np.trace(Q1 + Q2@Y.transpose()@Y) 

        @pymanopt.function.numpy(manifold_)
        def euclidean_gradient(Y):
            grad = 2*Q2@Y.transpose()
            return grad.transpose() 
                                
        @pymanopt.function.numpy(manifold_)
        def euclidean_hessian(Y,U):
            hess = 2*Q2@U.transpose()
            return hess.transpose()


    problem = Problem(manifold=manifold_, 
                      cost=cost,
                      euclidean_gradient=euclidean_gradient, 
                      euclidean_hessian=euclidean_hessian,
                      )

    optimizer = optimizers.TrustRegions(max_iterations=max_iter,use_rand=use_rand,min_gradient_norm=10**-9)
    print(n_anchor)
    if  Y_init is None:
        Y_init_ = manifold_.random_point()
    elif n_anchor > 1:
        if Y_init[0].shape[0] == d:
            print(1)
            Y_init_ = Y_init
        elif Y_init[0].shape[0] < d:
            print(2)
            Y_init_ = np.zeros((d,n))
            Y_init_[:Y_init[0].shape[0],:] = Y_init[1]
            Y_init_ = [A, Y_init_]
        else:
            print(3)
            if len(Y_init) == 2:
                U,S,V = np.linalg.svd(Y_init[1],full_matrices=False)
            else:
                U,S,V = np.linalg.svd(Y_init,full_matrices=False)
            Y_init_ = [A, U[:d,:d]@np.diag(S)[:d,:]@V/np.maximum(np.linalg.norm(U[:d,:d]@np.diag(S)[:d,:]@V,axis=0),10**-9)]
    else:
        if Y_init.shape[0] == d:
            print(1)
            Y_init_ = Y_init
        elif Y_init.shape[0] < d:
            print(2)
            Y_init_ = np.zeros((d,n))
            Y_init_[:Y_init.shape[0],:] = Y_init
        else:
            print(3)
            U,S,V = np.linalg.svd(Y_init,full_matrices=False)
            Y_init_ = U[:d,:d]@np.diag(S)[:d,:]@V/np.maximum(np.linalg.norm(U[:d,:d]@np.diag(S)[:d,:]@V,axis=0),10**-9)
    
    Y_star = optimizer.run(problem,initial_point=Y_init_).point

    return Y_star

In [265]:
W = np.diag(np.concatenate([np.ones(n_anchor),np.ones(n)]))
#Y_D10 = simple_IK(3,C,D,A_,n, n_anchor,max_iter=1000,W=W,Y_init=None)
#Y_D5 = simple_IK(5,C,D,A_anchor,n, n_anchor,max_iter=1000,W=W,Y_init=None) #[A_,Y_init])
Y_D4 = simple_IK(4,C,D,A_anchor,n, n_anchor,max_iter=2000,W=W,Y_init=None,use_rand =False)
Y_D3 = simple_IK(3,C,D,A_anchor,n, n_anchor,max_iter=3000,W=W,Y_init=Y_D4,use_rand =False)

14
(4, 14) (4, 30)
Optimizing...
                                            f: +1.766037e+00   |grad|: 4.877578e-01
(4, 14) (4, 30)
acc TR+   k:     1     num_inner:     0     f: +9.530972e-01   |grad|: 3.616913e-01   exceeded trust region
(4, 14) (4, 30)
acc       k:     2     num_inner:     0     f: +2.912319e-01   |grad|: 2.043692e-01   exceeded trust region
(4, 14) (4, 30)
acc       k:     3     num_inner:     2     f: +7.382182e-02   |grad|: 9.119783e-02   exceeded trust region
(4, 14) (4, 30)
REJ TR-   k:     4     num_inner:     3     f: +7.382182e-02   |grad|: 9.119783e-02   exceeded trust region
(4, 14) (4, 30)
acc TR+   k:     5     num_inner:     2     f: +2.998414e-02   |grad|: 4.075644e-02   exceeded trust region
(4, 14) (4, 30)
acc TR-   k:     6     num_inner:     2     f: +2.507624e-02   |grad|: 7.477214e-02   exceeded trust region
(4, 14) (4, 30)
acc TR+   k:     7     num_inner:     1     f: +4.864665e-03   |grad|: 1.236881e-02   exceeded trust region
(4, 14) (4, 30)

In [266]:
Y_star = Y_D3
e1 = np.array([1,0,0])
e2 = np.array([0,1,0])
e3 = np.array([0,0,1])

#X = direction_to_position(np.concatenate([Y_star[0][:3,:],U[:3,:3]@np.diag(S)[:3,:]@V],axis=1),C,D)
if len(Y_star) == 2:
    print(ANCHOR)
    X = direction_to_position(np.concatenate([Y_star[0],Y_star[1]],axis=1),C,D,data)
else:
    X = direction_to_position(Y_star,C,D,data)


R = np.eye(3)#rotation_matrix_from_vectors(norm(X[1,:]), e2)
R_ = np.eye(3)#rotation_matrix_from_vectors(norm(R@X[2,:]), e3).transpose()
Rz_flip = np.diag((1,1,1))
print(X)
print(np.linalg.norm(X[-2,:]@R.transpose()@R_.transpose()@Rz_flip-position))
print(np.linalg.norm((X[-1,:]-X[-2,:])/axis_length-direction))
print((X[-2,:]-X[-1,:])/np.linalg.norm((X[-2,:]-X[-1,:])))
#X = X@R.transpose()@Rz_flip
print(np.linalg.norm(X[-2,:]-position))

anchor
(3, 44) (16, 44) (44, 44)
[[ 0.00000000e+00  0.00000000e+00  1.57500000e-01]
 [ 2.00000000e-01  2.76356476e-11  1.57500000e-01]
 [-2.58150433e-27 -2.00000000e-01  1.57500000e-01]
 [-3.25705983e-09  3.23575760e-09  3.57500001e-01]
 [ 3.03685319e-06  5.19960838e-06  3.60000019e-01]
 [-1.72069491e-01  1.01941644e-01  3.59999979e-01]
 [-7.64507877e-05 -1.28848236e-04  5.64500000e-01]
 [-5.58008672e-05 -9.39728237e-05  7.64500008e-01]
 [-5.43495254e-05 -9.12804420e-05  7.80000024e-01]
 [-1.69473073e-02  1.99194009e-01  7.79966988e-01]
 [ 1.47917234e-04 -4.34899192e-05  9.64499971e-01]
 [ 6.06060086e-05 -1.78359509e-05  1.16449998e+00]
 [ 6.53358156e-05 -1.92268155e-05  1.17999999e+00]
 [ 5.65284090e-02  1.91845103e-01  1.17999999e+00]
 [-1.77100371e-08  1.76416018e-08  1.26099999e+00]
 [-2.10369668e-08  2.10341762e-08  1.46099999e+00]]
2.6342976897968822e-08
2.742558082040102e-08
[ 1.66346482e-08 -1.69628718e-08 -1.00000000e+00]
2.6342976897968822e-08


In [267]:
print(list_edges_anchor)
print('''


''')

print(A_anchor.transpose())

[('lbr_iiwa_joint_1', 'x'), ('lbr_iiwa_joint_1', 'y'), ('lbr_iiwa_joint_1', 'lbr_iiwa_joint_1_tilde'), ('lbr_iiwa_joint_1', 'lbr_iiwa_joint_7'), ('lbr_iiwa_joint_1', 'lbr_iiwa_joint_7_tilde'), ('x', 'y'), ('x', 'lbr_iiwa_joint_7'), ('x', 'lbr_iiwa_joint_7_tilde'), ('y', 'lbr_iiwa_joint_1_tilde'), ('y', 'lbr_iiwa_joint_7'), ('y', 'lbr_iiwa_joint_7_tilde'), ('lbr_iiwa_joint_1_tilde', 'x'), ('lbr_iiwa_joint_1_tilde', 'lbr_iiwa_joint_7'), ('lbr_iiwa_joint_1_tilde', 'lbr_iiwa_joint_7_tilde')]




[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]
 [ 1.30541272e-13  2.18324724e-12  1.00000000e+00]
 [ 1.42245428e-13  1.84828196e-12  1.00000000e+00]
 [-7.07106781e-01 -7.07106781e-01  0.00000000e+00]
 [-1.78336135e-01  2.14824897e-12  9.83969625e-01]
 [-1.51658302e-01  1.82690285e-12  9.88432982e-01]
 [ 0.00000000e+00  7.07106781e-01  7.07106781e-01]
 [ 1.28448647e-13  1.78336135e-01  9.8396962

In [268]:
data.oMi[-1].translation + pinocchio.SE3.act(data.oMi[-1],pinocchio.SE3(np.eye(3),np.array([0,0,axis_length]))).translation

array([3.29469210e-13, 4.81844886e-12, 2.72200000e+00])

In [269]:
position
#print(X)
A_anchor.transpose()

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 1.30541272e-13,  2.18324724e-12,  1.00000000e+00],
       [ 1.42245428e-13,  1.84828196e-12,  1.00000000e+00],
       [-7.07106781e-01, -7.07106781e-01,  0.00000000e+00],
       [-1.78336135e-01,  2.14824897e-12,  9.83969625e-01],
       [-1.51658302e-01,  1.82690285e-12,  9.88432982e-01],
       [ 0.00000000e+00,  7.07106781e-01,  7.07106781e-01],
       [ 1.28448647e-13,  1.78336135e-01,  9.83969625e-01],
       [ 1.40600073e-13,  1.51658302e-01,  9.88432982e-01],
       [ 7.07106781e-01,  0.00000000e+00, -7.07106781e-01],
       [ 1.59438068e-13,  2.66653384e-12,  1.00000000e+00],
       [ 1.68026203e-13,  2.18326736e-12,  1.00000000e+00]])

In [270]:
nx.get_node_attributes(G1,POS)
nx.get_edge_attributes(G1,DIST)

{('lbr_iiwa_joint_1', 'x'): 0.2,
 ('lbr_iiwa_joint_1', 'y'): 0.2,
 ('lbr_iiwa_joint_1', 'lbr_iiwa_joint_1_tilde'): 0.20000000000000004,
 ('lbr_iiwa_joint_1', 'lbr_iiwa_joint_2'): 0.20249999999999999,
 ('lbr_iiwa_joint_1', 'lbr_iiwa_joint_2_tilde'): 0.2846159693348155,
 ('lbr_iiwa_joint_1', 'lbr_iiwa_joint_7'): 1.1035,
 ('lbr_iiwa_joint_1', 'lbr_iiwa_joint_7_tilde'): 1.3034999999999999,
 ('x', 'y'): 0.28284271247461906,
 ('x', 'lbr_iiwa_joint_7'): 1.1214777082046448,
 ('x', 'lbr_iiwa_joint_7_tilde'): 1.3187540521264478,
 ('y', 'lbr_iiwa_joint_1_tilde'): 0.28284271247461906,
 ('y', 'lbr_iiwa_joint_7'): 1.1214777082051,
 ('y', 'lbr_iiwa_joint_7_tilde'): 1.3187540521268413,
 ('lbr_iiwa_joint_1_tilde', 'x'): 0.28284271247461906,
 ('lbr_iiwa_joint_1_tilde', 'lbr_iiwa_joint_2'): 0.0024999999999999467,
 ('lbr_iiwa_joint_1_tilde', 'lbr_iiwa_joint_2_tilde'): 0.20001562438970838,
 ('lbr_iiwa_joint_1_tilde', 'lbr_iiwa_joint_7'): 0.9034999999999999,
 ('lbr_iiwa_joint_1_tilde', 'lbr_iiwa_joint_7_til

In [271]:
def direction_to_position(Y,C,D,data):
    print(Y.shape, C.shape, D.shape)
    X = np.linalg.pinv(C@C.transpose())@C@D@Y.transpose()
    #print(X)
    X = X - X[0,:] + data.oMi[1].translation*scale
    
    #if X[0,0]<0:
      #  X = -X
    
    v = X[1,:] - X[0,:]
    R = rotation_matrix_from_vectors(v/np.linalg.norm(v),e1/np.linalg.norm(e1))
    X = (X - X[0,:])@R.transpose() + X[0,:]

    v = X[2,:] - X[0,:]
    R = rotation_matrix_from_vectors(v/np.linalg.norm(v),-e2/np.linalg.norm(e2))
    X = (X - X[0,:])@R.transpose() + X[0,:]

    v = X[2,:] - X[0,:]
    R = np.diag((1,1,1))
    X = (X - X[0,:])@R.transpose() + X[0,:]

    return X

def rotation_matrix_from_vectors(vec1, vec2):
    """ Find the rotation matrix that aligns vec1 to vec2
    :param vec1: A 3d "source" vector
    :param vec2: A 3d "destination" vector
    :return mat: A transform matrix (3x3) which when applied to vec1, aligns it with vec2.
    """
    a, b = (vec1 / np.linalg.norm(vec1)).reshape(3), (vec2 / np.linalg.norm(vec2)).reshape(3)
    v = np.cross(a, b)
    c = np.dot(a, b)
    s = np.linalg.norm(v)
    kmat = np.array([[0, -v[2], v[1]], [v[2], 0, -v[0]], [-v[1], v[0], 0]])
    rotation_matrix = np.eye(3) + kmat + kmat.dot(kmat) * ((1 - c) / (s ** 2))
    return rotation_matrix